# El ecosistema JuMP para optimización matemática

##  Ejemplo de motivación: Número mínimo de pasaportes necesarios para visitar todos los países sin necesidad de VISA.

Utilizamos los datos provenientes de https://www.passportindex.org/. "Become a global citizen".

In [1]:
;git clone https://github.com/ilyankou/passport-index-dataset.git

fatal: destination path 'passport-index-dataset' already exists and is not an empty directory.


In [2]:
using DelimitedFiles
data = readdlm(joinpath("passport-index-dataset\\legacy\\2019-11-23","passport-index-matrix.csv"),',')
cntr = data[2:end,1]
vf = (x ->  x == -1 || x == 3 ? 1 : 0).(data[2:end,2:end])

199×199 Array{Int64,2}:
 1  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  1  0  1  0  1  0  1  0  0  0     0  0  0  0  0  0  1  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  1  0  1  1  1  0  1  0  1  0     1  1  0  1  1  1  1  1  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  1
 0  1  0  1  0  1  0  0  0  1  0  1  0  …  0  1  0  0  0  1  1  1  0  0  1  1
 0  1  0  1  0  1  1  1  0  1  0  1  0     1  1  0  1  1  1  1  1  0  0  0  0
 0  1  0  0  0  1  1  1  0  0  0  1  0     0  0  0  1  1  0  0  0  0  0  0  0
 0  1  0  1  0  1  1  1  1  1  0  1  0     1  1  0  1  1  1  1  1  0  0  0  0
 0  1  0  1  0  1  1  1  0  1  0  1  0     1  1  0  1  1  1  1  1  0  0  0  0
 0  1  0  0  0  1  0  0  0  0  1  1  0  …  0  0  0  0  1  1  0  0  0  0  0  0
 0  1  0  1  0  1  0  0  0  1  0  1  0     1  1  1  1  0  1  1  0  0  0  1  1
 0  0  0  0  0  0  0  0  0  0  0  1  1  

### (Constrained) Mathematical Optimization and JuMP

$$ \begin{align*} \min_{x,y} &&\quad \sum_{\operatorname{cntr} \;\in\; \operatorname{World}} \operatorname{pass}_{\,\operatorname{cntr}} \\ \text{s.t.}&&\quad \sum_{\operatorname{cntr} \;\in\; \operatorname{World}}\operatorname{vf}(\operatorname{cntr},\operatorname{dst}) \cdot \operatorname{pass}_{\,\operatorname{cntr}} &\geq 1\quad &\quad& \forall \; \operatorname{dst} \;\in \; \operatorname{World}\\ &&\operatorname{pass}_{\,\operatorname{cntr}} &\in \{0,1\}&\quad& \forall \; \operatorname{cntr}\in \; \operatorname{World}. \end{align*} $$

In [3]:
using JuMP, GLPK
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, pass[1:length(cntr)], Bin)
@constraint(model, [j=1:length(cntr)], sum( vf[i,j]*pass[i] for i in 1:length(cntr)) >= 1)
@objective(model, Min, sum(pass))
JuMP.optimize!(model)
print(JuMP.objective_value(model)," passports: ",join(cntr[findall(JuMP.value.(pass) .== 1)],", "))

24.0 passports: Afghanistan, Angola, Australia, Austria, China, Comoros, Congo, Eritrea, Gambia, Georgia, Hong Kong, India, Iraq, Kenya, Madagascar, Maldives, North Korea, Papua New Guinea, Seychelles, Singapore, Somalia, Tunisia, United Arab Emirates, United States

## Ejemplo paso a paso: Resolviendo un Problema Lineal

Resolvamos el siguiente problema lineal de 2 variables

$$
\begin{align*}
\max_{x,y} \quad x + 2y \\
\text{s.t.} \quad x + y\leq 1 \\
0\leq x, y \leq 1
\end{align*}
$$

Cargamos JuMP, MathOptInterface (MOI) y GLPK (GNU LP/MIP solver -gratuito-):

In [4]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities
using GLPK # Loading the GLPK module for using its solver

Construir un objeto tipo modelo (que contiene variables, restricciones, opciones del solver, etc):

In [5]:
model = Model(with_optimizer(GLPK.Optimizer,msg_lev = 4));

Definir las variables del modelo $0\leq x, y \leq 1$:

In [6]:
@variable(model, 0 <= x <= 1)
@variable(model, 0 <= y <= 1)

y

Añadir restricción al modelo $x + y \leq 1$:

In [7]:
@constraint(model, x + y <= 1)

x + y <= 1.0

Añadir función objetivo al modelo $\max x + 2y$:

In [8]:
@objective(model, Max, x + 2y)

x + 2 y

Veamos cómo está quedando el modelo

In [9]:
model

A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x, y

El modelo está completo y ya hemos seleccionado un solver al comienzo. Solo queda llamar a la función `Optimize` para resolverlo

In [10]:
JuMP.optimize!(model)

GLPK Simplex Optimizer, v4.64
1 row, 2 columns, 2 non-zeros
*     0: obj = -0.000000000e+000 inf =  0.000e+000 (2)
*     2: obj =  2.000000000e+000 inf =  0.000e+000 (0)
OPTIMAL LP SOLUTION FOUND


Podemos chequear el estado de la optimización:
- ¿Obtuvimos valores óptimos para el primal?
- ¿Se terminó el problema?
- ¿Obtuvimos un punto factible?
- ¿Qué pasa con el Dual?

In [11]:
@show JuMP.has_values(model)
@show JuMP.termination_status(model) == MOI.OPTIMAL
@show JuMP.primal_status(model) == MOI.FEASIBLE_POINT
@show JuMP.dual_status(model) == MOI.FEASIBLE_POINT

JuMP.has_values(model) = true
JuMP.termination_status(model) == MOI.OPTIMAL = true
JuMP.primal_status(model) == MOI.FEASIBLE_POINT = true
JuMP.dual_status(model) == MOI.FEASIBLE_POINT = true


true

In [12]:
display(typeof(MOI.FEASIBLE_POINT))

Enum MathOptInterface.ResultStatusCode:
NO_SOLUTION = 0
FEASIBLE_POINT = 1
NEARLY_FEASIBLE_POINT = 2
INFEASIBLE_POINT = 3
INFEASIBILITY_CERTIFICATE = 4
NEARLY_INFEASIBILITY_CERTIFICATE = 5
REDUCTION_CERTIFICATE = 6
NEARLY_REDUCTION_CERTIFICATE = 7
UNKNOWN_RESULT_STATUS = 8
OTHER_RESULT_STATUS = 9

Recuperemos los valores que nos importan

In [13]:
@show JuMP.value(x)              # Old syntax: getvalue(x)
@show JuMP.value(y)              # Old syntax: getvalue(y)
@show JuMP.objective_value(model);       # Old syntax: getobjectivevalue(model)

JuMP.value(x) = 0.0
JuMP.value(y) = 1.0
JuMP.objective_value(model) = 2.0


Podemos también ponerle nombre a alguna restricción para llamarla más adelante (o pedir calcular su multiplicador, eliminarla, etc)

In [14]:
model = Model(with_optimizer(GLPK.Optimizer, msg_lev = 0))
@variable(model, 0 <= x <= 1)
@variable(model, 0 <= y <= 1)
@constraint(model, inequality, x + y <= 1)     # <=============== constraint can be referenced later as "inequality"
@objective(model, Max, x + 2y)
JuMP.optimize!(model)
@show JuMP.termination_status(model) == MOI.OPTIMAL

JuMP.termination_status(model) == MOI.OPTIMAL = true


true

In [15]:
λ = JuMP.dual(inequality)

-1.0

In [16]:
JuMP.delete(model, inequality)
JuMP.optimize!(model)
@show JuMP.termination_status(model) == MOI.OPTIMAL
@show JuMP.objective_value(model);

JuMP.termination_status(model) == MOI.OPTIMAL = true
JuMP.objective_value(model) = 3.0


# Restricciones no lineales

In [17]:
using Ipopt

model = Model(with_optimizer(Ipopt.Optimizer))
@variable(model, x)
@variable(model, y)

@NLobjective(model, Min, (1-x)^2 + 100(y-x^2)^2)
@NLconstraint(model, exp(x)+sin(x) <=0)

JuMP.optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        1
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

# Optimización Cónica (Otro tipo de restricciones)

¿Por qué Optimización Cónica?
- Dualidad similar al caso de programación lineal.
- Algoritmos más rápidos y estables
    - Se evitan problemas con no diferenciabilidad, se explota la forma dual-primal, teoría fuerte en barreras y métodos de punto interior.
    - Cambios en la industria en el 2018:
        1. Knitro versión 11.0 añade soporte para restricciones SOCP.
        2. Mosek version 9.0 desecha expresiones y formulaciones basadas en funciones por formulaciones puramente cónicas (lineal, SOCP, SOCP rotado, SDP, exp,etc)

## Optimización Cónica

Los problemas de programación cónica son problemas de optimización convexos en los cuales la función objetivo es minimizazda sobre la intersección de subespacios afines y conos convexos. Un ejemplo de formulación cónica de un problema es:

\begin{equation}
\min_{ x \in \mathbb{R}^n} a_0 ^\top x + b_0
\end{equation}

tal que:
$$A_i x + b_i \in \mathcal{C}_i  \quad \text{for } i = 1 \dotso m$$

El problema dual correspondiente es:

$$ \max_{y_1, \dotso , y_m} - \sum_{i = 1}^{m} b_i ^T y_i + b_0$$

tal que:
$$ a_0 - \sum_{i = 1}^{m} A_{i}^{T} y_{i} = 0 $$
$$ y_i \in \mathcal{C}_i^*$$

Donde cada $\mathcal{C}_i$ es un cono cerrado convexo y $\mathcal{C}_i^*$ es el cono dual.

### Cono de Segundo Orden

El Cono de Segundo Orden (o Cono de Lorentz) de dimensión $n$ es de la forma:

$$ Q^n = \{ (t,x) \in \mathbb{R}^n: t \ge \lVert x \rVert_2 \} $$

Un Cono de Segundo Orden rotado en $\pi/4$ en el plano  $(x_1,x_2)$ es llamado 
Cono de Segundo Orden Rotado y es de la forma:

$$ Q^n_r = \{ (t,u,x) \in \mathbb{R}^n: 2tu \ge \lVert x \rVert_2, t, u \ge 0 \} $$


Estos conos se representan en `JuMP` usando conjuntos `MOI`, `SecondOrderCone` y `RotatedSecondOrderCone`

#### Ejemplo: Proyección Euclideana en un Hiperplano

Para un punto dado $u_0$ y un conjunto $K$, nos referiamos al punto $u \in K$ que es lo más cercano a $u_0$ como la proyección de $u_0$ en $K$. La proyección de $u_0$ en un hiperplano $K = \{ u : p'\cdot u = q \}$ está dado por:

$$ \begin{align}
& \displaystyle\min_{x \in \mathbb{R}^n} & \lVert u - u_0 \rVert \\\\
& \text{s.t. } & p'\cdot u = q 
\end{align}$$

Podemos modelar esto como un problema de optimización cónica:

$$ \begin{align}
& \displaystyle\min & t \\\\
& \text{s.t. } & p'\cdot u = q \\\\
&  \quad & (t, u - u_0) \in Q^{n+1}
\end{align}$$

Si lo transformamos identificando cada uno de los términos,

$$ \begin{align}
x & = (t,u)\\\\
a_0 & = e_1\\\\
b_0 & = 0\\\\
A_1 & =(0,p)\\\\
b_1 & = -q \\\\
C_1 & = \mathbb{R}\\\\
A_2 & = 1\\\\
b_2 &= -(0,u_0)\\\\
C_2 &= Q^{n+1}
\end{align}$$

Entonces, el problema dual está dado por:

$$ \begin{align}
& \displaystyle\max & y_1 + (0,u_0)^\top y_2 \\\\
& \text{s.t. } & e_1 -(0,p)^\top y_1 - y_2 = 0 \\\\
&  \quad & y_1 \in \mathbb{R}\\\\
& \quad & y_2 \in Q^{n+1}
\end{align}$$

En primer lugar, cargamos los paquetes `JuMP`, `LinearAlgebra`, `Random` y el solver `ECOS`.

In [18]:
using JuMP
using ECOS
using LinearAlgebra
using Random
Random.seed!(2020);

Démosno algunos valores aleatorios para los inputs: $u_0$, $p$ y $q$

In [19]:
u0 = rand(10)
p = rand(10)
q = rand();

Podemos escribir el modelo como:

In [20]:
model = Model(optimizer_with_attributes(ECOS.Optimizer, "printlevel" => 0))
@variable(model, u[1:10])
@variable(model, t)
@objective(model, Min, t)
@constraint(model, [t, (u - u0)...] in SecondOrderCone())
@constraint(model, u' * p == q)
optimize!(model)


ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -0.000e+00  +1e+01  4e-01  3e-04  1e+00  6e+00    ---    ---    1  1  - |  -  - 
 1  +3.134e-01  +3.200e-01  +1e+00  6e-02  3e-05  1e-01  7e-01  0.8932  1e-03   2  1  2 |  0  0
 2  +1.376e+00  +1.404e+00  +7e-02  5e-03  2e-06  4e-02  5e-02  0.9690  5e-02   2  2  2 |  0  0
 3  +1.415e+00  +1.415e+00  +8e-04  6e-05  2e-08  4e-04  6e-04  0.9890  1e-04   1  1  1 |  0  0
 4  +1.415e+00  +1.415e+00  +9e-06  6e-07  2e-10  5e-06  7e-06  0.9890  1e-04   1  1  1 |  0  0
 5  +1.415e+00  +1.415e+00  +1e-07  7e-09  2e-12  5e-08  7e-08  0.9890  1e-04   1  1  1 |  0  0
 6  +1.415e+00  +1.415e+00  +1e-09  8e-11  3e-14  6e-10  8e-10  0.9890  1e-04   2  1  1 |  0  0

OPTIMAL (within feastol=7.5e-11, reltol=7.4e-10, abstol=1.1e-09).
Runtime: 0.000117 seconds.



In [21]:
@show objective_value(model);
@show value.(u);

objective_value(model) = 1.4149915748070703
value.(u) = [0.03080667950164791, 0.5687503071622262, 0.4982756380955075, 0.32732944869183017, 0.5755950055187378, 0.117335178956295, 0.021419168446740567, 0.5510291353919837, -0.24224068275971017, 0.053604926321581634]


El problema dual se puede calcular como:

In [22]:
e1 = [1.0, zeros(10)...]
dual_model = Model(optimizer_with_attributes(ECOS.Optimizer, "printlevel" => 0))
@variable(dual_model, y1 <= 0.0)
@variable(dual_model, y2[1:11])
@objective(dual_model, Max, q * y1 + dot(vcat(0.0, u0), y2))
@constraint(dual_model, e1 - [0.0, p...] .* y1 - y2 .== 0.0)
@constraint(dual_model, y2 in SecondOrderCone())
optimize!(dual_model)


ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -0.000e+00  +5e+00  4e-01  4e-01  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  -8.629e+00  -2.721e+00  +5e-01  4e-01  4e-01  7e+00  2e-01  0.8934  1e-03   2  1  1 |  0  0
 2  -2.387e+00  -2.455e+00  +2e-01  9e-02  5e-02  4e-02  5e-02  0.7977  5e-02   2  2  2 |  0  0
 3  -1.467e+00  -1.426e+00  +8e-03  5e-03  2e-03  5e-02  5e-03  0.9655  7e-02   2  2  2 |  0  0
 4  -1.416e+00  -1.415e+00  +9e-05  5e-05  3e-05  5e-04  6e-05  0.9890  1e-04   2  1  1 |  0  0
 5  -1.415e+00  -1.415e+00  +1e-06  6e-07  3e-07  6e-06  7e-07  0.9890  1e-04   2  1  1 |  0  0
 6  -1.415e+00  -1.415e+00  +1e-08  7e-09  3e-09  6e-08  7e-09  0.9890  1e-04   3  1  1 |  0  0

OPTIMAL (within feastol=6.7e-09, reltol=8.1e-09, abstol=1.1e-08).
Runtime: 0.000123 seconds.



In [23]:
@show objective_value(dual_model);

objective_value(dual_model) = 1.4149916455792486


Para ver más sobre el tema: [Click aquí](https://danpereda.github.io/post/conicopt/)